## Automated Extraction of OSM Building Footprints in Gaza Using Overpass API

This Python script implements an automated spatial data extraction pipeline for building footprints in the Gaza Strip using the OpenStreetMap (OSM) Overpass API. The script divides the Gaza region into **30 spatially uniform grid cells** (organized in a 5-row by 6-column layout) and performs an **iterative data retrieval** for each cell. This strategy enables manageable querying of OSM data by subregion, thereby overcoming common API limitations related to large bounding boxes, server timeout constraints, and memory usage.

---

### Methodological Overview

#### 1. **Library Initialization**
The script uses the following Python packages:
- `overpy`: Interfaces with the Overpass API.
- `geojson`: Constructs GeoJSON feature collections.
- `tqdm`: Provides a progress bar during looping.

#### 2. **Spatial Grid Definition**
The Gaza Strip is bounded approximately by:
- **Latitude**: 31.22° N to 31.62° N  
- **Longitude**: 34.22° E to 34.55° E  

The area is subdivided into **30 rectangular zones** by calculating uniform steps in latitude and longitude. Each zone is identified (Z1 through Z30) and stored as a bounding box in a dictionary.

#### 3. **Overpass Query Execution**
For each zone, the script constructs a query using Overpass QL that selects all building features:

```ql
way["building"](min_lat, min_lon, max_lat, max_lon);


In [7]:
!pip install overpy geojson tqdm

import overpy
import geojson
from tqdm import tqdm

api = overpy.Overpass()

# Define bounds
min_lat, max_lat = 31.22, 31.62
min_lon, max_lon = 34.22, 34.55
lat_step = (max_lat - min_lat) / 5
lon_step = (max_lon - min_lon) / 6

zones = {}

zone_id = 1
for i in range(5):  # rows
    for j in range(6):  # cols
        zones[f"Z{zone_id}"] = {
            "min_lat": min_lat + i * lat_step,
            "max_lat": min_lat + (i + 1) * lat_step,
            "min_lon": min_lon + j * lon_step,
            "max_lon": min_lon + (j + 1) * lon_step
        }
        zone_id += 1

# Loop through and query
for zone_name, bbox in tqdm(zones.items()):
    query = f"""
    [out:json][timeout:300];
    (
      way["building"]({bbox['min_lat']},{bbox['min_lon']},{bbox['max_lat']},{bbox['max_lon']});
    );
    (._;>;);
    out body;
    """
    try:
        result = api.query(query)
        features = []
        for way in result.ways:
            coords = [(float(node.lon), float(node.lat)) for node in way.nodes]
            if coords[0] != coords[-1]:
                coords.append(coords[0])
            features.append(geojson.Feature(
                geometry=geojson.Polygon([coords]),
                properties={"id": way.id, **way.tags, "zone": zone_name}
            ))
        collection = geojson.FeatureCollection(features)
        with open(f"{zone_name}.geojson", "w") as f:
            geojson.dump(collection, f)
        print(f"{zone_name}: {len(features)} buildings saved.")
    except Exception as e:
        print(f"{zone_name} failed: {e}")

  3%|▎         | 1/30 [00:12<06:01, 12.46s/it]

Z1: 27546 buildings saved.


  7%|▋         | 2/30 [00:19<04:13,  9.06s/it]

Z2: 19417 buildings saved.


 10%|█         | 3/30 [00:21<02:36,  5.78s/it]

...

Z28: 0 buildings saved.


 97%|█████████▋| 29/30 [04:20<00:06,  6.26s/it]

Z29: 13998 buildings saved.


100%|██████████| 30/30 [04:26<00:00,  8.87s/it]

Z30: 13988 buildings saved.


In [8]:
!zip gaza_30zones.zip Z*.geojson

  adding: Z10.geojson (deflated 85%)
  adding: Z11.geojson (deflated 68%)
  adding: Z12.geojson (deflated 86%)
  adding: Z13.geojson (deflated 7%)
  adding: Z14.geojson (deflated 88%)
  adding: Z15.geojson (deflated 87%)
  adding: Z16.geojson (deflated 87%)
  adding: Z17.geojson (deflated 87%)
  adding: Z18.geojson (deflated 86%)
  adding: Z19.geojson (deflated 7%)
  adding: Z1.geojson (deflated 87%)
  adding: Z20.geojson (deflated 7%)
  adding: Z21.geojson (deflated 87%)
  adding: Z22.geojson (deflated 87%)
  adding: Z23.geojson (deflated 88%)
  adding: Z24.geojson (deflated 87%)
  adding: Z25.geojson (deflated 7%)
  adding: Z26.geojson (deflated 7%)
  adding: Z27.geojson (deflated 7%)
  adding: Z28.geojson (deflated 7%)
  adding: Z29.geojson (deflated 88%)
  adding: Z2.geojson (deflated 87%)
  adding: Z30.geojson (deflated 88%)
  adding: Z3.geojson (deflated 87%)
  adding: Z4.geojson (deflated 86%)
  adding: Z5.geojson (deflated 87%)
  adding: Z6.geojson (deflated 7%)
  adding: Z7.ge

## Generation of a 30-Zone Spatial Grid over the Gaza Strip (KML Output)

This Python script constructs a 30-zone spatial grid overlay for the Gaza Strip and exports it in **KML format** for use in geospatial platforms such as **QGIS**, **Google Earth**, or web-based map viewers. The purpose of this grid is to spatially subdivide the Gaza Strip into manageable units for mapping, data collection, or Overpass API querying.

---

### Methodological Overview

#### 1. **Grid Definition**
The bounding box for the Gaza Strip is defined as:
- **Latitude**: 31.22° N to 31.62° N  
- **Longitude**: 34.22° E to 34.55° E  

This area is subdivided into a grid of **5 rows** (north–south) and **6 columns** (west–east), resulting in **30 equal rectangular zones**. The dimensions of each zone are:
- **Latitude step**: 0.08°
- **Longitude step**: 0.055°

Each grid cell is labeled sequentially from **Z1** to **Z30**, starting from the northwest corner and proceeding row-wise.

#### 2. **KML Geometry Construction**
Using the `simplekml` library:
- Each zone is represented as a rectangular **polygon** with a transparent red fill (`#66ff0000`) and a visible border.
- A labeled **point** is placed at the centroid of each zone for visual reference.

#### 3. **Zone Metadata Storage**
For every zone, the script records:
- Bounding coordinates (`MinLat`, `MaxLat`, `MinLon`, `MaxLon`)
- Grid position (`Row`, `Column`)
- Zone ID (`Z1` to `Z30`)

This metadata is stored in a Python list (`zone_data`) for further analysis or tabular export (e.g., as a CSV or Pandas DataFrame).

#### 4. **File Output**
The output file is saved as:
```bash
gaza_grid_30zones.kml


In [9]:
!pip install simplekml pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for simplekml: filename=simplekml-1.3.6-py3-none-any.whl size=65860 sha256=33f95682fd9c6f49eababfb37a218bf14b88b0f4516578c8157a0ff01f75858c
  Stored in directory: /root/.cache/pip/wheels/72/3e/80/c3e5c354c3cbe62d8c5e4fb63d9e7cdccc7f93399997ae465f
Successfully built simplekml


In [10]:
import simplekml
import pandas as pd

min_lat, max_lat = 31.22, 31.62
min_lon, max_lon = 34.22, 34.55
lat_step = (max_lat - min_lat) / 5
lon_step = (max_lon - min_lon) / 6

kml = simplekml.Kml()
zone_data = []
zone_id = 1

for row in range(5):
    for col in range(6):
        min_lat_zone = min_lat + row * lat_step
        max_lat_zone = min_lat + (row + 1) * lat_step
        min_lon_zone = min_lon + col * lon_step
        max_lon_zone = min_lon + (col + 1) * lon_step
        zone_name = f"Z{zone_id}"

        pol = kml.newpolygon(
            name=zone_name,
            outerboundaryis=[
                (min_lon_zone, min_lat_zone),
                (min_lon_zone, max_lat_zone),
                (max_lon_zone, max_lat_zone),
                (max_lon_zone, min_lat_zone),
                (min_lon_zone, min_lat_zone)
            ]
        )
        pol.style.polystyle.color = '66ff0000'
        pol.style.linestyle.width = 1.5
        center_lat = (min_lat_zone + max_lat_zone) / 2
        center_lon = (min_lon_zone + max_lon_zone) / 2
        kml.newpoint(name=zone_name, coords=[(center_lon, center_lat)])

        zone_data.append({
            "Zone": zone_name,
            "MinLat": min_lat_zone,
            "MaxLat": max_lat_zone,
            "MinLon": min_lon_zone,
            "MaxLon": max_lon_zone,
            "Row": row + 1,
            "Column": col + 1
        })
        zone_id += 1

kml.save("gaza_grid_30zones.kml")
print("KML file saved as gaza_grid_30zones.kml")


KML file saved as gaza_grid_30zones.kml
